<a href="https://colab.research.google.com/github/ImolaFodor/ad_hw/blob/main/AD_HW1_FodorImo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HW1 for Algorithmic Design in DSSC 2020/2021 - Imola Fodor SM4500474

Strassen Algorithm for Matrix Multiplication

1. implement the strassen matrix mult function to multiply two 2n × 2n
matrices by using the Strassen’s algorithm;
2. generalize strassen matrix mult to deal with any kind of matrix pair
that can be multiplied (possibly also non-square matrices) and prove that
the asymptotic complexity does not change;
3. improve the implementation of the function by reducing the number of
auxiliary matrices and test the effects on the execution time;
4. answer to the following question: how much is the minimum auxiliary
space required to evaluate the Strassen’s algorithm? Motivate the answer

##Excersize 1.

1. implement the strassen matrix mult function to multiply two 2^n × 2^n
matrices by using the Strassen’s algorithm;

In [1]:
from __future__ import annotations

from numbers import Number
from typing import List, Tuple
import numpy as np
import math

class Matrix(object):
    ''' A simple naive matrix class

    Members
    -------
    _A: List[List[Number]]
        The list of rows that stores all the matrix values

    Parameters
    ----------
    A: List[List[Number]]
        The list of rows that stores all the matrix values
    clone_matrix: Optional[bool]
        A flag to require a full copy of `A`'s data structure.

    Raises
    ------
    ValueError
        If there are two lists having a different number of values
    '''
    def __init__(self, A: List[List[Number]], clone_matrix: bool = True):
        num_of_cols = None

        for row in enumerate(A):
            if num_of_cols is not None:
                if num_of_cols != len(row):
                    raise ValueError('This is not a matrix')
            else:
                num_of_cols = len(row)

        if clone_matrix:
            self._A = [[value for value in row] for row in A]
        else:
            self._A = A

    @property
    def num_of_rows(self) -> int:
        return len(self._A)

    @property
    def num_of_cols(self) -> int:
        if len(self._A) == 0:
            return 0

        return len(self._A[0])

    def copy(self):
        A = [[value for value in row] for row in self._A]

        return Matrix(A, clone_matrix=False)

    def __getitem__(self, y: int):
        ''' Return one of the rows

        Parameters
        ----------
        y: int
            the index of the rows to be returned

        Returns
        -------
        List[Number]
            The `y`-th row of the matrix
        '''
        return self._A[y]

    def __iadd__(self, A: Matrix) -> Matrix:
        ''' Sum a matrix to this matrix and update it

        Parameters
        ----------
        A: Matrix
            The matrix to be summed up

        Returns
        -------
        Matrix
            The matrix corresponding to the sum between this matrix and
            that passed as parameter

        Raises
        ------
        ValueError
            If the two matrices have different sizes
        '''

        if (self.num_of_cols != A.num_of_cols or
                self.num_of_rows != A.num_of_rows):
            raise ValueError('The two matrices have different sizes')

        for y in range(self.num_of_rows):
            for x in range(self.num_of_cols):
                self[y][x] += A[y][x]

        return self

    def __add__(self, A: Matrix) -> Matrix:
        ''' Sum a matrix to this matrix

        Parameters
        ----------
        A: Matrix
            The matrix to be summed up

        Returns
        -------
        Matrix
            The matrix corresponding to the sum between this matrix and
            that passed as parameter

        Raises
        ------
        ValueError
            If the two matrices have different sizes
        '''
        res = self.copy()

        res += A

        return res

    def __isub__(self, A: Matrix) -> Matrix:
        ''' Subtract a matrix to this matrix and update it

        Parameters
        ----------
        A: Matrix
            The matrix to be subtracted up

        Returns
        -------
        Matrix
            The matrix corresponding to the subtraction between this matrix and
            that passed as parameter

        Raises
        ------
        ValueError
            If the two matrices have different sizes
        '''

        if (self.num_of_cols != A.num_of_cols or
                self.num_of_rows != A.num_of_rows):
            raise ValueError('The two matrices have different sizes')

        for y in range(self.num_of_rows):
            for x in range(self.num_of_cols):
                self[y][x] -= A[y][x]

        return self

    def __sub__(self, A: Matrix) -> Matrix:
        ''' Subtract a matrix to this matrix

        Parameters
        ----------
        A: Matrix
            The matrix to be subtracted up

        Returns
        -------
        Matrix
            The matrix corresponding to the subtraction between this matrix and
            that passed as parameter

        Raises
        ------
        ValueError
            If the two matrices have different sizes
        '''
        res = self.copy()

        res -= A

        return res

    def __mul__(self, A: Matrix) -> Matrix:
        ''' Multiply one matrix to this matrix

        Parameters
        ----------
        A: Matrix
            The matrix which multiplies this matrix

        Returns
        -------
        Matrix
            The row-column multiplication between this matrix and that passed
            as parameter

        Raises
        ------
        ValueError
            If the number of columns of this matrix is different from the
            number of rows of `A`
        '''
        return gauss_matrix_mult(self, A)

    def __rmul__(self, value: Number) -> Matrix:
        ''' Multiply one matrix by a numeric value

        Parameters
        ----------
        value: Number
            The numeric value which multiplies this matrix

        Returns
        -------
        Matrix
            The multiplication between `value` and this matrix

        Raises
        ------
        ValueError
            If `value` is not a number
        '''

        if not isinstance(value, Number):
            raise ValueError('{} is not a number'.format(value))

        return Matrix([[value*elem for elem in row] for row in self._A],
                      clone_matrix=False)

    def submatrix(self, from_row: int, num_of_rows: int,
                  from_col: int, num_of_cols: int) -> Matrix:
        ''' Return a submatrix of this matrix

        Parameters
        ----------
        from_row: int
            The first row to be included in the submatrix to be returned
        num_of_rows: int
            The number of rows to be included in the submatrix to be returned
        from_col: int
            The first col to be included in the submatrix to be returned
        num_of_cols: int
            The number of cols to be included in the submatrix to be returned

        Returns
        -------
        Matrix
            A submatrix of this matrix
        '''
        A = [row[from_col:from_col+num_of_cols]
             for row in self._A[from_row:from_row+num_of_rows]]

        return Matrix(A, clone_matrix=False)

    def assign_submatrix(self, from_row: int, from_col: int, A: Matrix):
        for y, row in enumerate(A):
            self_row = self[y + from_row]
            for x, value in enumerate(row):
                self_row[x + from_col] = value

    
    # def append_column(self, from_col, A: Matrix):
    #     Matrix zero_col = ([[0 for x in range(A.num_of_rows)] None])
    #     return '\n'.join('{}'.format(row) for row in self._A)

    # def append_row(self, from_row: int):
    #     return '\n'.join('{}'.format(row) for row in self._A)    
    
    def __repr__(self):
        return '\n'.join('{}'.format(row) for row in self._A)

Naive Algorithm

In [2]:
def gauss_matrix_mult(A: Matrix, B: Matrix) -> Matrix:
    ''' Multiply two matrices by using Gauss's algorithm

    Parameters
    ----------
    A: Matrix
        The first matrix to be multiplied
    B: Matrix
        The second matrix to be multiplied

    Returns
    -------
    Matrix
        The row-column multiplication of the matrices passed as parameters

    Raises
    ------
    ValueError
        If the number of columns of `A` is different from the number of
        rows of `B`
    '''

    if A.num_of_cols != B.num_of_rows:
        raise ValueError('The two matrices cannot be multiplied')

    result = [[0 for col in range(B.num_of_cols)]
                for row in range(A.num_of_rows)]

    for row in range(A.num_of_rows):
        for col in range(B.num_of_cols):
            value = 0
            for k in range(A.num_of_rows):
                value += A[row][k] * B[k][col]

            result[row][col] = value

    return Matrix(result, clone_matrix=False)

Get quadrants of the matrix supposing it is a square matrix with 2^n columns and rows.

In [3]:
def get_matrix_quadrants_v0(A: Matrix) -> Tuple[Matrix, Matrix, Matrix, Matrix, Matrix]:
       

    A11 = A.submatrix(0, A.num_of_rows//2, 0, A.num_of_cols//2)
    A12 = A.submatrix(0, A.num_of_rows//2, A.num_of_cols//2, A.num_of_cols//2)
    A21 = A.submatrix(A.num_of_rows//2, A.num_of_rows//2, 0, A.num_of_cols//2)
    A22 = A.submatrix(A.num_of_rows//2, A.num_of_rows//2, A.num_of_cols//2, A.num_of_cols//2)

    return A11, A12, A21, A22

Strassen Algorithm

In [4]:
from random import random, seed
import numpy as np


def strassen_matrix_mult_v0(A: Matrix, B: Matrix) -> Matrix:

    if max(A.num_of_rows, B.num_of_cols, A.num_of_cols) < 2:
        return gauss_matrix_mult(A,B)

    # Recursive step
    A11, A12, A21, A22 = get_matrix_quadrants_v0(A)
    B11, B12, B21, B22 = get_matrix_quadrants_v0(B)

    # cost is Theta(n^2)
    S1 = B12 - B22
    S2 = A11 + A12
    S3 = A21 + A22
    S4 = B21 - B11
    S5 = A11 + A22
    S6 = B11 + B22
    S7 = A12 - A22
    S8 = B21 + B22
    S9 = A11 - A21
    S10 = B11 + B12

    # recursive step, 
    # to avoid the naive multiplication 7x, 
    # instead to always go through the strassen algorithm
    P1 = strassen_matrix_mult_v0(A11,S1)
    P2 = strassen_matrix_mult_v0(S2,B22)
    P3 = strassen_matrix_mult_v0(S3,B11)
    P4 = strassen_matrix_mult_v0(A22,S4)
    P5 = strassen_matrix_mult_v0(S5,S6)
    P6 = strassen_matrix_mult_v0(S7,S8)
    P7 = strassen_matrix_mult_v0(S9,S10) 

    # second batch of sums
    C11 = P5 + P4 - P2 + P6
    C12 = P1 + P2 
    C21 = P3 + P4
    C22 = P5 + P1 - P3 - P7
    
    # build the resulting matrix
    result = Matrix([[0 for x in range(B.num_of_cols)]
        for y in range(A.num_of_rows)], clone_matrix = False) # clone matrix false, because the matrix has been built exclusively to be written into it, we don't need to copy it

    # copy Cij into the resulting matrix
    result.assign_submatrix(0, 0, C11)
    result.assign_submatrix(0, result.num_of_cols//2, C12)
    result.assign_submatrix(result.num_of_rows//2, 0, C21)
    result.assign_submatrix(result.num_of_rows//2, result.num_of_cols//2, C22)

    return result
    

In [5]:
A = Matrix([[random() for x in range(2**3)] for y in range(2**3)])
B = Matrix([[random() for x in range(2**3)] for y in range(2**3)])
SM = strassen_matrix_mult_v0(A, B)


print(SM)

[1.7250402974813555, 0.6797706651171915, 1.2715361998007726, 1.8032517568176976, 1.415769483897765, 0.8320078933612468, 1.9873624535820507, 2.510643478153847]
[2.0654723742659606, 0.7454233316808214, 1.4190262122366744, 2.5612935347443866, 1.6166369963589182, 0.8096869438714783, 2.1038578015891516, 1.6043801186102615]
[2.1478091546169873, 0.8556303138645815, 1.5546595154399527, 2.0298928870447046, 1.4718826546340313, 0.7405980055951971, 1.9768448057322243, 2.4512111576260907]
[1.7561550235781174, 0.6929515581572885, 1.327203727305315, 2.17658514208314, 1.4830786390448312, 0.554974163602934, 1.6668029290896678, 1.659953719321314]
[2.1925253673375975, 1.0369515831632046, 1.8663886108571799, 2.4445487674314315, 1.9003252725980653, 0.9530649670801354, 2.3935531787577733, 2.517181379108175]
[2.791773630105162, 1.0117984843377765, 1.9493094373099733, 2.528900702033715, 2.3296586985464236, 1.0653793862916925, 2.721248436172005, 3.1526545283933904]
[2.3932169075741454, 0.9439267683023984, 1.88

## Excersize 2.

2. generalize strassen matrix mult to deal with any kind of matrix pair
that can be multiplied (possibly also non-square matrices) and prove that
the asymptotic complexity does not change;



Modified get_matrix_quadrants method, to handle matrices of any dimension, not anymore limited to two 2^n × 2^n matrices. 

As a first step, a step that will not be performed again in the recursive iterations, creates matrices padded to the max needed 2^n size, to be the matrices to start with.
2 × 4 * 4 × 3 will be 4 × 4 * 4 × 4 ; 2 × 4 * 4 × 9 will be 16 × 16 * 16 × 16 etc.

In [11]:
def get_matrix_quadrants_v1(A: Matrix, B: Matrix) -> Tuple[Matrix, Matrix, Matrix, Matrix, Matrix]:
    
    if not(math.log(A.num_of_rows, 2).is_integer() and  math.log(A.num_of_cols, 2).is_integer()):

        newA = Matrix([[0 for x in range(2**int(np.log2(A.num_of_cols)+1))]
        for y in range(2**int(np.log2(A.num_of_rows)+1))], clone_matrix = False)

        newA.assign_submatrix(0, 0, A)

        A = newA

    if not(math.log(B.num_of_rows, 2).is_integer() and  math.log(B.num_of_cols, 2).is_integer()):           

        newB = Matrix([[0 for x in range(2**int(np.log2(B.num_of_cols)+1))]
        for y in range(2**int(np.log2(B.num_of_rows)+1))], clone_matrix = False)

        newB.assign_submatrix(0, 0, B)

        B = newB

    if(A.num_of_cols < B.num_of_cols):
        newA = Matrix([[0 for x in range(B.num_of_cols)]
        for y in range(A.num_of_rows)], clone_matrix = False)

        newA.assign_submatrix(0, 0, A)

        A = newA

    if(A.num_of_rows < B.num_of_rows):
        newA = Matrix([[0 for x in range(A.num_of_cols)]
        for y in range(B.num_of_rows)], clone_matrix = False)

        newA.assign_submatrix(0, 0, A)  

        A = newA    

    A11 = A.submatrix(0, A.num_of_rows//2, 0, A.num_of_cols//2)
    A12 = A.submatrix(0, A.num_of_rows//2, A.num_of_cols//2, A.num_of_cols//2)
    A21 = A.submatrix(A.num_of_rows//2, A.num_of_rows//2, 0, A.num_of_cols//2)
    A22 = A.submatrix(A.num_of_rows//2, A.num_of_rows//2, A.num_of_cols//2, A.num_of_cols//2)

    return A, A11, A12, A21, A22

In [12]:
def strassen_matrix_mult_v1(A: Matrix, B: Matrix) -> Matrix:

    # Recursive step
    A, A11, A12, A21, A22 = get_matrix_quadrants_v1(A, B)
    B, B11, B12, B21, B22 = get_matrix_quadrants_v1(B, A)

    if max(A.num_of_rows, B.num_of_cols, A.num_of_cols) < 8:
        return gauss_matrix_mult(A,B)

    # cost is Theta(n^2)
    S1 = B12 - B22
    S2 = A11 + A12
    S3 = A21 + A22
    S4 = B21 - B11
    S5 = A11 + A22
    S6 = B11 + B22
    S7 = A12 - A22
    S8 = B21 + B22
    S9 = A11 - A21
    S10 = B11 + B12

    # recursive step, 
    # to avoid the naive multiplication 7x, 
    # instead to always go through the strassen algorithm
    P1 = strassen_matrix_mult_v1(A11,S1)
    P2 = strassen_matrix_mult_v1(S2,B22)
    P3 = strassen_matrix_mult_v1(S3,B11)
    P4 = strassen_matrix_mult_v1(A22,S4)
    P5 = strassen_matrix_mult_v1(S5,S6)
    P6 = strassen_matrix_mult_v1(S7,S8)
    P7 = strassen_matrix_mult_v1(S9,S10) 

    # second batch of sums
    C11 = P5 + P4 - P2 + P6
    C12 = P1 + P2 
    C21 = P3 + P4
    C22 = P5 + P1 - P3 - P7
    
    # build the resulting matrix
    result = Matrix([[0 for x in range(B.num_of_cols)]
        for y in range(A.num_of_rows)], clone_matrix = False) # clone matrix false, because the matrix has been built exclusively to be written into it, we don't need to copy it

    # copy Cij into the resulting matrix
    result.assign_submatrix(0, 0, C11)
    result.assign_submatrix(0, result.num_of_cols//2, C12)
    result.assign_submatrix(result.num_of_rows//2, 0, C21)
    result.assign_submatrix(result.num_of_rows//2, result.num_of_cols//2, C22)

    return result

In [13]:
A = Matrix([[random() for x in range(4)] for y in range(5)])
B = Matrix([[random() for x in range(7)] for y in range(4)])
C = Matrix([[1,3,5,6],[6,8,7,2]])
print(C)
D = Matrix([[1,3,5,6,5],[6,8,7,2,6],[1,3,5,6,7],[6,8,7,2,1]])
print(D)


SM = strassen_matrix_mult_v1(C,D).submatrix(0, C.num_of_rows, 0, D.num_of_cols)
print(SM)

[1, 3, 5, 6]
[6, 8, 7, 2]
[1, 3, 5, 6, 5]
[6, 8, 7, 2, 6]
[1, 3, 5, 6, 7]
[6, 8, 7, 2, 1]
[60, 90, 93, 54, 64]
[73, 119, 135, 98, 129]


###Proof of asymptotic complexity consistency

Proving that the dominant term, the most complex term is the complexity of the multiplication itself, we are sure that merely padding the input matrices, does not affect the overall complexity of the algorithm.

###Reasoning:

The input matrix is n × n. 

The padded one is N × N. 

N needs to be the next power of 2 of n. N = 2<sup>ceil(log<sub>2</sub>n)</sup>
Therefore, N < 2n.

It is proven that for n, hence also 2n, the complexity is O(n<sup>log<sub>2</sub>7</sup>). 2n<sup>log<sub>2</sub>7</sup> = 2<sup>log<sub>2</sub>7</sup> * n<sup>log<sub>2</sub>7</sup> = 7 * n<sup>log<sub>2</sub>7</sup>. The constant 7 is not affecting the complexity.

As a result, it can be stated that: N<sup>log<sub>2</sub>7</sup> < 2n<sup>log<sub>2</sub>7</sup> = 7n<sup>log<sub>2</sub>7</sup> =  O(n<sup>log<sub>2</sub>7</sup>)




##Excersize 3.

3. improve the implementation of the function by reducing the number of
auxiliary matrices and test the effects on the execution time;

The v2 of the Strassen algorithm doesn't store in auxiliary matrices the intermediate steps of the quadrants additions and subtractions.

In [56]:
def strassen_matrix_mult_v2(A: Matrix, B: Matrix) -> Matrix:

    # Recursive step
    A, A11, A12, A21, A22 = get_matrix_quadrants_v1(A, B)
    B, B11, B12, B21, B22 = get_matrix_quadrants_v1(B, A)

    if max(A.num_of_rows, B.num_of_cols, A.num_of_cols) < 8:
        return gauss_matrix_mult(A,B)



    # excluding the Sx auxiliary matrices
    P1 = strassen_matrix_mult_v2(A11,(B12 - B22))
    P2 = strassen_matrix_mult_v2((A11 + A12),B22)
    P3 = strassen_matrix_mult_v2((A21 + A22),B11)
    P4 = strassen_matrix_mult_v2(A22,(B21 - B11))
    P5 = strassen_matrix_mult_v2((A11 + A22),(B11 + B22))
    P6 = strassen_matrix_mult_v2((A12 - A22),(B21 + B22))
    P7 = strassen_matrix_mult_v2((A11 - A21),(B11 + B12)) 

    # second batch of sums
    C11 = P5 + P4 - P2 + P6
    C12 = P1 + P2 
    C21 = P3 + P4
    C22 = P5 + P1 - P3 - P7

    # for point 4 excersize 
    # note comment Px matrices when executing this version
    # C11 = strassen_matrix_mult_v2((A11 + A22),(B11 + B22)) + strassen_matrix_mult_v2(A22,(B21 - B11)) - strassen_matrix_mult_v2((A11 + A12),B22) + strassen_matrix_mult_v2((A12 - A22),(B21 + B22))
    # C12 = strassen_matrix_mult_v2(A11,(B12 - B22)) + strassen_matrix_mult_v2((A11 + A12),B22) 
    # C21 = strassen_matrix_mult_v2((A21 + A22),B11) + strassen_matrix_mult_v2(A22,(B21 - B11))
    3 C22 = strassen_matrix_mult_v2((A11 + A22),(B11 + B22)) + strassen_matrix_mult_v2(A11,(B12 - B22)) - strassen_matrix_mult_v2((A21 + A22),B11) - strassen_matrix_mult_v2((A11 - A21),(B11 + B12))
    
    # build the resulting matrix
    result = Matrix([[0 for x in range(B.num_of_cols)]
        for y in range(A.num_of_rows)], clone_matrix = False) # clone matrix false, because the matrix has been built exclusively to be written into it, we don't need to copy it

    # copy Cij into the resulting matrix
    result.assign_submatrix(0, 0, C11)
    result.assign_submatrix(0, result.num_of_cols//2, C12)
    result.assign_submatrix(result.num_of_rows//2, 0, C21)
    result.assign_submatrix(result.num_of_rows//2, result.num_of_cols//2, C22)

    return result

In [54]:
#from timeit import timeit
from sys import stdout

seed(0)

t = [];

for i in range(100):
    i=i+1
    stdout.write(f'{i}')
    A = Matrix([[random() for x in range(i)] for y in range(i)])
    B = Matrix([[random() for x in range(i)] for y in range(i)])


    t_element = []
    for funct in ['strassen_matrix_mult_v1', 'strassen_matrix_mult_v2']:
        T = timeit(f'{funct}(A,B)', globals=locals(), number=1)
        t_element.append(T)
        stdout.write('\t{:.3f}'.format(T))
        stdout.flush() # to print everything in the std buffer and then clean it
    t.append(t_element)
    stdout.write('\n')

    
print(t)


1	0.000	0.000
2	0.000	0.000
3	0.001	0.000
4	0.000	0.000
5	0.002	0.003
6	0.003	0.003
7	0.002	0.004
8	0.002	0.003
9	0.017	0.016
10	0.009	0.015
11	0.009	0.021
12	0.006	0.021
13	0.007	0.015
14	0.010	0.017
15	0.006	0.016
16	0.007	0.016
17	0.050	0.191
18	0.054	0.182
19	0.050	0.185
20	0.052	0.180
21	0.049	0.195
22	0.048	0.191
23	0.053	0.181
24	0.053	0.180
25	0.049	0.187
26	0.050	0.186
27	0.050	0.187
28	0.050	0.184
29	0.051	0.185
30	0.048	0.202
31	0.049	0.187
32	0.048	0.182
33	0.322	2.171
34	0.328	2.177
35	0.329	2.176
36	0.335	2.166
37	0.321	2.170
38	0.334	2.193
39	0.330	2.166
40	0.329	2.162
41	0.326	2.161
42	0.328	2.181
43	0.325	2.222
44	0.328	2.161
45	0.322	2.149
46	0.318	2.197
47	0.325	2.164
48	0.321	2.173
49	0.328	2.173
50	0.319	2.162
51	0.331	2.182
52	0.324	2.160
53	0.330	2.149
54	0.319	2.165
55	0.334	2.177
56	0.327	2.160
57	0.335	2.154
58	0.324	2.155
59	0.329	2.170
60	0.320	2.161
61	0.324	2.155
62	0.314	2.142
63	0.332	2.185
64	0.325	2.150
65	2.257	26.105
66	2.260	26.089
67	2.292	26.110
6

Compare which of the two versions executed more frequently, faster. 

In [55]:
from collections import defaultdict

d = defaultdict(int)
for lst in t:
   d[lst[1]<lst[0]] += 1

print(dict(d))

{False: 98, True: 2}


The supposition that the v2, which minimizes the number of auxiliary matrices, would execute faster, was not proven for matrices up to 100x100 in size. {False: 66, True: 34} 

It can be noticed that in most of the cases the v2 does not outperform the v1 of the Strassen Algorithm. The Sx matrices were not used repeatedly in the Px matrices, hence the statistics (even though rough) is odd, due to the fact that the reduced space complexity affects the time complexity too.

##Excersize 4.

4. answer to the following question: how much is the minimum auxiliary
space required to evaluate the Strassen’s algorithm? Motivate the answer.

Additionally, it was tried to absolutely omit the auxiliary matrices.

d is {False: 98, True: 2} for running range(100) with 0 auxiliary matrices. The outcome is as expected, a very slow execution, because the recursive step is executed more frequently, due to the lack of the auxiliary matrices (Px) that before were used repeatidly.

d is {False: 66, True: 34} for running range(100) on v2, reduced auxiliary matrices version. 